# GPU env

In [5]:
import numpy as np
import datetime
import tensorflow as tf

## Ex1. GPU environment test 1

In [2]:
a=tf.constant([1.,2.,3.,4.,5.,6.],shape=[2,3],name='a')
b=tf.constant([1.,2.,3.,4.,5.,6.],shape=[3,2],name='b')
c=tf.matmul(a,b)

sess=tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


## Ex2. GPU environment test 2

In [ ]:
with tf.device('/gpu:1'):
    a=tf.constant([1.,2.,3.,4.,5.,6.],shape=[2,3],name='a')
    b=tf.constant([1.,2.,3.,4.,5.,6.],shape=[3,2],name='b')
    c=tf.matmul(a,b)

    sess=tf.Session(config=tf.ConfigProto(log_device_placement=True))
    print(sess.run(c))

## Ex3. multi-GPU environment

In [4]:
c=[]
for d in ['/gpu:0','/gpu:1']:
    with tf.device(d):
        a=tf.constant([1.,2.,3.,4.,5.,6.],shape=[2,3])
        b=tf.constant([1.,2.,3.,4.,5.,6.],shape=[3,2])
        c=tf.matmul(a,b)
    with tf.device('/cpu:0'):
        sum=tf.add_n(c)
sess=tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

## Ex4. GPU test code

In [6]:
def matpow(M,n):
    if n<1:
        return M
    else:
        return tf.matmul(M,matpow(M,n-1))


In [9]:
A=np.random.rand(10000,10000).astype('float32')
B=np.random.rand(10000,10000).astype('float32')

n=10
c1=[]
c2=[]

In [10]:
# use 1 gpu
with tf.device('/gpu:0'):
    a=tf.constant(A)
    b=tf.constant(B)
    c1.append(matpow(a,n))
    c1.append(matpow(b,n))
with tf.device('/cpu:0'):
    sum=tf.add_n(c1)

t1_1=datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(sum)
t2_1=datetime.datetime.now()


# use 2 gpu
with tf.device('/gpu:0'):
    a=tf.constant(A)
    c2.append(matpow(a,n))
    
with tf.device('/gpu:1'):
    b=tf.constant(B)
    c2.append(matpow(b,n))
    
with tf.device('/cpu:0'):
    sum=tf.add_n(c2)

t1_2=datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(sum)
t2_2=datetime.datetime.now()


## print time for computation
print("Single GPU computaiton time: "+str(t2_1-t1_1))
print("Multi GPU computaiton time: "+str(t2_2-t1_2))
